In [1]:
import matplotlib.pyplot as plt
import numpy as np

def plot_gaussian(datum):
    mu = datum[0]
    A = datum[1]
    sigma = datum[2]
    x = np.arange(mu - 100, mu + 100, 0.1)
    y = A*np.exp(-0.5*np.square(x - mu)/(sigma*sigma))
    plt.plot(x, y, linestyle = "dashed")

def plot_bounds(datum,J):
    datum = datum[1:]
    start, end = datum[0], datum[1]
    plt.plot()
    plt.vlines([start,end],J[0]/2 + 10,J[1]/2 - 10, colors = ['red', 'black'])
    plt.hlines(J[0]/4 + J[1]/4, start, end)

def plot_biexp_curve(t, kappa, rho):
    return np.exp(-t/kappa) - np.exp(-t/rho)

def plot_biexp(datum):
    datum = datum[1:]
    
    start, end = datum[0], datum[1]
    baseline, amp, kappa, rho = datum[2], datum[3], datum[4], datum[5]
    x = np.arange(start, end, 0.1)
    y = amp*plot_biexp_curve(x - start, kappa, rho) + baseline
    plt.plot(x, y, linestyle = "dashed")

def plot_second_order_cpx(x, A,B,C, param_lambda, param_theta):

    return np.exp(-x*param_lambda)*(A*np.cos(0.5*x*param_theta) + B*np.sin(0.5*x*param_theta)) + C

def plot_second_order(datum):
    datum = datum[1:]
    start, end, _ = datum[0], datum[1], datum[2]
    param_lambda, param_theta = datum[3], datum[4]
    A,B,C = datum[5], datum[6], datum[7]
    x = np.arange(start, end, 0.1)
    y = plot_second_order_cpx(x - start, A, B, C, param_lambda, param_theta)
    plt.plot(x,y, linestyle = "dashed")

In [2]:


# f = open('data/retrace.csv')
# tokens = [e.split(",") for e in f.readlines()]
# x = [int(e[0]) for e in tokens]
# y = [int(e[1]) for e in tokens]
# handle = plt.plot(x,y)

def get_trace_file(filename, I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    #maxy = max([e[1] for e in tokens])
    x = [e[0] for e in tokens]
    y = [e[1] for e in tokens]
    return (x,y)

def get_detect_file(filename,I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    #tokens = [token for token in tokens if I[0] <= token[0] <= I[1]]
    x = [e[0] for e in tokens]
    y = [e[1] for e in tokens]
    c = ['r' if e[2] == 1 else ('g' if e[2] == 0 else 'b') for e in tokens]
    return (x,y,c)

def get_event_file(filename,I):
    f = open(filename, 'r')
    tokens = [[float(x) for x in e.split(",")] for e in f.readlines()]
    return tokens


In [3]:
def create_trace_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    #plt.ylim(J)
    plt.plot(data[0],data[1])

def create_event_plot(ax,data,I,J):
    plt.subplot(ax)
    plt.xlim(I)
    plt.ylim(J)
    plt.subplot(ax)
    for datum in data:
        plot_bounds(datum,J)
        plot_second_order(datum)
    #plt.scatter(x1,y,marker='|',color='r')
    #plt.scatter(x,y,marker='.',color='g')
    #plt.scatter(x2,y,marker='|',color='b')

def create_plot(I):
    data = [get_trace_file('Saves/output_baselined.csv',I),
            get_trace_file('Saves/output_smoothed.csv',I),
            get_trace_file('Saves/output_diff.csv',I),
            get_trace_file('Saves/output_cuts.csv',I),
            get_trace_file('Saves/output_simulated.csv',I),
            get_event_file('Saves/output_pulses.csv',I)
    ]
    fig = plt.figure(figsize = (14,12))
    maxy = max(data[0][1]) + 10
    ax = plt.subplot(2,1,1)
    create_trace_plot(ax,data[0],I,(0,maxy))
    create_trace_plot(ax,data[1],I,(0,maxy))
    #create_trace_plot(ax,data[2],I,(0,maxy))
    #create_trace_plot(ax,data[2],I,(0,maxy))
    create_event_plot(ax,data[5],I,(0,maxy))

    return fig,ax



In [4]:
from ipywidgets import interact, widgets

def func(delta, size):
    I = (delta,delta + size)
    fig,ax = create_plot(I)

interact(func,
    delta = widgets.IntSlider(min=0, max=30000, step=100, value=5000),
    size = widgets.IntSlider(min=100, max=25000, step=100, value=400)
)

interactive(children=(IntSlider(value=5000, description='delta', max=30000, step=100), IntSlider(value=400, de…

<function __main__.func(delta, size)>